<a href="https://colab.research.google.com/github/96jonesa/CSE-517-Project/blob/main/scaffolding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Imports

In [57]:
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

#GRU

##GRU
This is just a wrapper around nn.GRU for the sake of consistency. Used in the Price Encoder, day-level SMI Encoder, and temporal SMI Encoder.

In [58]:
class GRU(nn.Module):
    def __init__(self, input_size, hidden_size, batch_first=False):
        super(GRU, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.batch_first = batch_first

        self.gru = nn.GRU(input_size, hidden_size, batch_first=self.batch_first)

    def forward(self, input):
        output, hn = self.gru(input)
        return output, hn

#Self-Attention

##LinearAttention
The attention mechanism used in Feng et. al. Used in the Price Encoder, day-level SMI Encoder, and temporal SMI Encoder. Given input $h$, returns
$q_t = \sum_{i=t-T}^T \beta_i h_i$ where $\beta_i = \dfrac{\exp\left( u^T \tanh \left( W h_i + b \right) \right)}{\sum_{k=t-T}^t \exp\left( u^T \tanh \left( W h_k + b \right) \right)}$.

In [59]:
# attention weights are softmax(u^T tanh(W input + b)) where W is learned parameter matrix, u is a learned parameter vector, and b is a learned offset

class LinearAttention(nn.Module):
    def __init__(self, input_size, intermediate_size, weights_size):
        super(LinearAttention, self).__init__()
        self.input_size = input_size
        self.intermediate_size = intermediate_size
        self.weights_size = weights_size

        self.linear_1 = nn.Linear(self.input_size, self.intermediate_size, bias=True)
        self.linear_2 = nn.Linear(self.intermediate_size, self.weights_size, bias=False)
        self.tanh = nn.Tanh()
        self.softmax = nn.Softmax(dim=2)

    def forward(self, input):
        intermediate = self.tanh(self.linear_1(input))
        attention_weights = self.softmax(self.linear_2(intermediate))
        attention_weights = attention_weights.permute(0, 2, 1)
        output_features = torch.bmm(attention_weights, input)

        return output_features

#Blending

##Blend
Applies a learned bilinear transformation to the left and right vectors, then inputs the result to a ReLU non-linearity. Used to obtain Multi-Modal Encodings from Price Encodings and temporal SMI Encodings. Given Price Encodings $q_t$ and temporal SMI Encodings $c_t$, returns
$x_t = \mathcal{B} \left( c_t, q_t \right) = \text{ReLU} \left( q_t^T W c_t + b \right)$.

In [60]:
# output is ReLU(left^T W right + b) where W is a learned paramater matrix
# and b is a learned bias

class Blend(nn.Module):
    def __init__(self, left_size, right_size, output_size):
        super(Blend, self).__init__()
        self.left_size = left_size
        self.right_size = right_size
        self.output_size = output_size

        self.bilinear = nn.Bilinear(self.left_size, self.right_size, output_size, bias=True)
        self.relu = nn.ReLU()
    
    def forward(self, left, right):
        output = self.relu(self.bilinear(left, right))

        return output

#Single-Headed Graph Attention Network (SGAT)

##SharedLinear
This is just a wrapper around nn.Linear for the sake of consistency. Used to apply a shared linear transformation to all inputs of an SGAT layer. Under current implementation, this should be applied before passing inputs to SGAT.

In [61]:
# need shared learned parameter matrix W to multiply against each input vector

class SharedLinear(nn.Module):
    def __init__(self, input_size, output_size):
        super(SharedLinear, self).__init__()
        self.input_size = input_size
        self.output_size = output_size
        
        self.linear = nn.Linear(input_size, output_size, bias=False)
    
    def forward(self, input):
        output = self.linear(input)

        return output

##SGAT
A single-headed GAT layer. A shared linear transform $W$ is applied to all the nodes *before* passing them as input to this module (by passing them as input to a SharedLinear layer), then a shared self-attention mechanism is applied to each node $i$ in its immediate neighborhood $\mathcal{N}_i$. For each node $j\in \mathcal{N}_i$, normalized attention coefficients $\alpha_{i,j}$ are computed to represent the importance of the relations between stocks $i$ and $j$. That is,
$\alpha_{i,j} = \dfrac{\exp ( \text{LeakyReLU} ( a_w^T [ W x_i 
\oplus W x_j ] ) )}{\sum_{k\in \mathcal{N}_i} \exp ( \text{LeakyReLU} ( a_w^T [ W x_i \oplus W x_k ] ) )}$
where $\oplus$ denotes concatenation and $a_w$ is a learned parameter matrix. An updated feature vector $z_i$ for the $i$-th stock is computed by applying these attention weights to the linearly transformed multi-modal feature vectors of all of the stocks in $\mathcal{N}_i$

In [62]:
# merge code with MGAT code to form general case GAT code

class SGAT(nn.Module):
    def __init__(self, input_size, weights_size, leakyrelu_slope=0.01):
        super(SGAT, self).__init__()
        self.input_size = input_size
        self.weights_size = weights_size
        self.leakyrelu_slope = leakyrelu_slope
        
        self.linear = nn.Linear(2 * input_size, weights_size, bias=False)
        self.leakyrelu = nn.LeakyReLU(self.leakyrelu_slope)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, input, neighborhoods, index):
        stock = input[index]
        neighborhood = neighborhoods[index]
        stack_stock = stock.expand(len(neighborhood), stock.shape[0])
        stack_neighbors = input[neighborhood]
        cat_stock = torch.cat((stack_stock, stack_neighbors), dim=1)
        print(cat_stock)
        attention_weights = self.softmax(self.leakyrelu(self.linear(cat_stock)))  # check this
        output_features = torch.mm(attention_weights.T, stack_neighbors)  # check this

        return output_features

#MAN-SF

In [63]:
class MANSF(nn.Module):
    def __init__(self, T, num_stocks, gru_hidden_size, attn_inter_size, use_embed_size,
                 blend_size, gat_1_inter_size, gat_2_inter_size, leakyrelu_slope, elu_alpha, U):
        super(MANSF, self).__init__()
        self.T = T
        self.num_stocks = num_stocks
        self.gru_hidden_size = gru_hidden_size
        self.attn_inter_size = attn_inter_size
        self.use_embed_size = use_embed_size
        self.blend_size = blend_size
        self.gat_1_inter_size = gat_1_inter_size
        self.gat_2_inter_size = gat_2_inter_size
        self.leakyrelu_slope = leakyrelu_slope
        self.elu_alpha = elu_alpha
        self.U = U

        self.gru_p = GRU(3, gru_hidden_size, batch_first=True)
        self.gru_m = GRU(use_embed_size, gru_hidden_size, batch_first=True)
        self.gru_s = GRU(gru_hidden_size, gru_hidden_size, batch_first=True)
        self.attn_p = LinearAttention(gru_hidden_size, attn_inter_size, 1)
        self.attn_m = LinearAttention(gru_hidden_size, attn_inter_size, 1)
        self.attn_s = LinearAttention(gru_hidden_size, attn_inter_size, 1)
        self.blend = Blend(use_embed_size, use_embed_size, blend_size)
        self.shared_linears_1 = []
        for u in range(U):
            self.shared_linears_1.append(SharedLinear(blend_size, gat_1_inter_size))
        self.shared_linears_2 = []
        for u in range(U):
            self.shared_linears_2.append(SharedLinear(U * gat_1_inter_size, gat_2_inter_size))
        self.mgat_1 = []
        for u in range(U):
            sgats = []
            for i in range(num_stocks):
                sgats.append(SGAT(gat_1_inter_size, 1, leakyrelu_slope=leakyrelu_slope))
            self.mgat_1.append(sgats)
            #self.mgat_1.append(SGAT(gat_1_inter_size, 1, leakyrelu_slope=leakyrelu_slope))
        self.mgat_2 = []
        for u in range(U):
            sgats = []
            for i in range(num_stocks):
                sgats.append(SGAT(gat_2_inter_size, 1, leakyrelu_slope=leakyrelu_slope))
            self.mgat_2.append(sgats)
            #self.mgat_2.append(SGAT(gat_2_inter_size, 1, leakyrelu_slope=leakyrelu_slope))
        self.sigmoid = nn.Sigmoid()
        self.elu = nn.ELU(elu_alpha)
        self.final_linear = nn.Linear(U * gat_2_inter_size, 1, bias=True)

    # p is price data tensor of shape (num_stocks, T, 3), for the day under consideration
    #
    # m is smi data list of tensors of shape (num_stocks, K, use_embed_size) of length T,
    #       where K is the number of tweets for the given stock on the day under consideration
    #
    # neighorhoods is a list of adjacency lists, where each stock is indexed with the same
    #       indices they have in p and m
    #
    # TODO: tensorize day-level smi
    # TODO: tensorize sgat 
    def forward(self, p, m, neighborhoods):
        ## price encoding
        h_p, _ = self.gru_p(p)
        q = self.attn_p(h_p)

        ## smi encoding (day level)
        r = torch.zeros(self.num_stocks, 0, self.gru_hidden_size)
        for t in range(self.T):
            h_m, _ = self.gru_m(m[t])
            r_t = self.attn_m(h_m)
            r = torch.cat((r, r_t), 1)

        ## smi encoding (aggregate)
        h_s, _ = self.gru_s(r)
        c = self.attn_s(h_s)

        ## blending
        x = self.blend(q, c)

        ## reshaping (eliminating superfluous dimension)
        x = x.view(x.shape[0], x.shape[2])

        ## first gat layer
        #  first head
        shared_linear = self.shared_linears_1[0]
        Wx = shared_linear(x)
        #sgat = self.mgat_1[0]
        sgat = self.mgat_1[0][0]
        z = sgat(Wx, neighborhoods, 0)
        z = self.elu(z)

        for i in range(1, self.num_stocks):
            sgat = self.mgat_1[0][i]  # this is the fix to the proceeding TODO
            z_i = sgat(Wx, neighborhoods, i)  # TODO: use fresh SGAT for each stock
            z_i = self.elu(z_i)
            z = torch.cat((z, z_i), 0)

        #  remaining heads
        for u in range(1, self.U):
            shared_linear = self.shared_linears_1[u]
            Wx = shared_linear(x)
            #sgat = self.mgat_1[u]
            sgat = self.mgat_1[u][0]
            z_u = sgat(Wx, neighborhoods, 0)
            z_u = self.elu(z_u)

            for i in range(1, self.num_stocks):
                sgat = self.mgat_1[u][i]
                z_u_i = sgat(Wx, neighborhoods, i)
                z_u_i = self.elu(z_u_i)
                z_u = torch.cat((z_u, z_u_i), 0)
            
            z = torch.cat((z, z_u), 1)
        
        ## second gat layer
        #  first head
        shared_linear = self.shared_linears_2[0]
        Wx = shared_linear(z)
        #sgat = self.mgat_2[0]
        sgat = self.mgat_2[0][0]
        new_z = sgat(Wx, neighborhoods, 0)
        new_z = self.sigmoid(new_z)

        for i in range(1, self.num_stocks):
            sgat = self.mgat_2[0][i]
            new_z_i = sgat(Wx, neighborhoods, i)
            new_z_i = self.sigmoid(new_z_i)
            new_z = torch.cat((new_z, new_z_i), 0)

        #  remaining heads
        for u in range(1, self.U):
            shared_linear = self.shared_linears_2[u]
            Wx = shared_linear(z)
            #sgat = self.mgat_2[u]
            sgat = self.mgat_2[u][0]
            new_z_u = sgat(Wx, neighborhoods, 0)
            new_z_u = self.sigmoid(new_z_u)

            for i in range(1, self.num_stocks):
                sgat = self.mgat_2[u][i]
                new_z_u_i = sgat(Wx, neighborhoods, i)
                new_z_u_i = self.sigmoid(new_z_u_i)
                new_z_u = torch.cat((new_z_u, new_z_u_i), 0)
            
            new_z = torch.cat((new_z, new_z_u), 1)
        
        ## final layer
        y = self.sigmoid(self.final_linear(new_z))

        ## return result
        return y

#Real Data Testing

In [64]:
!pip3 install --quiet "tensorflow-hub>=0.7.0"
!pip3 install --quiet seaborn

In [65]:
from absl import logging

import tensorflow.compat.v1 as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import re
import seaborn as sns
import json
import itertools
import pandas as pd
from tqdm import tqdm_notebook

In [66]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder/2" #@param ["https://tfhub.dev/google/universal-sentence-encoder/2", "https://tfhub.dev/google/universal-sentence-encoder-large/3"]

In [67]:
tf.disable_v2_behavior()
tf.compat.v1.disable_eager_execution()

Instructions for updating:
non-resource variables are not supported in the long term


In [68]:
stocknet_dataset_filepath = './stocknet-dataset-master'

In [69]:
preprocessed_prices_filepath = stocknet_dataset_filepath + '/price/preprocessed'
preprocessed_tweets_filepath = stocknet_dataset_filepath + '/tweet/preprocessed'

company_to_price_df = {}
company_to_tweets = {}

for filename in os.listdir(preprocessed_prices_filepath):
    with open(preprocessed_prices_filepath + '/' + filename) as file:
        company_name = filename.split('.')[0]
        
        # Not enough data for GMRE
        if company_name == 'GMRE':
            continue
        df = pd.read_csv(file, sep='\t')
        df.columns = ['date', 'open', 'high', 'low', 'close', 'adjust_close', 'volume']
        company_to_price_df[company_name] = df.dropna()

for filename in tqdm_notebook(os.listdir(preprocessed_tweets_filepath)):
    company_name = filename.split('.')[0]
    dates_to_tweets = {}
    for tweet_filename in os.listdir(preprocessed_tweets_filepath + '/' + filename):
        with open(preprocessed_tweets_filepath + '/' + filename + '/' + tweet_filename) as file:
            list_of_tweets = []
            for line in file:
                tweet_json = json.loads(line)
                list_of_tweets.append(tweet_json)
            dates_to_tweets[tweet_filename] = list_of_tweets
    company_to_tweets[company_name] = dates_to_tweets

FileNotFoundError: ignored

In [ ]:
#print(company_to_tweets.keys())
#print(dates_to_tweets.keys())
print(company_to_tweets['AAPL']['2015-10-02'][0])

In [ ]:
# Reduce logging output.
logging.set_verbosity(logging.ERROR)
tf.get_logger().setLevel(logging.ERROR)
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

# Import the Universal Sentence Encoder's TF Hub module
def embed_useT(module):
    with tf.Graph().as_default():
        sentences = tf.placeholder(tf.string)
        embed = hub.Module(module)
        embeddings = embed(sentences)
        session = tf.train.MonitoredSession()
    return lambda x: session.run(embeddings, {sentences: x})
embed_fn = embed_useT(module_url)

In [ ]:
# Generate embeddings
for company in tqdm_notebook(company_to_tweets.keys()):
  for date in company_to_tweets[company].keys():
    messages = []
    for j in range(len(company_to_tweets[company][date])):
      messages.append(' '.join(company_to_tweets[company][date][j]['text']))
    message_embeddings = embed_fn(messages)
    for k in range(len(company_to_tweets[company][date])):
      company_to_tweets[company][date][k]['embedding'] = list(message_embeddings[k])

In [ ]:
# Create date mapping
date_universe = set()
for company in company_to_price_df.keys():
    date_universe = date_universe.union(set(company_to_price_df[company].date))
for company in company_to_tweets.keys():
    date_universe = date_universe.union(set(company_to_tweets[company].keys()))
date_universe = sorted(list(date_universe))
index_to_date = {i-5:d for i,d in enumerate(date_universe)}
date_to_index = {d:i-5 for i,d in enumerate(date_universe)}

In [ ]:
# Calculate dimensions for tensor
n_stocks = len(company_to_tweets.keys())
n_days = len(date_universe)
max_tweets = 0
for c,d in itertools.product(company_to_tweets.keys(), date_universe):
    if d in company_to_tweets[c]:
        max_tweets = max(max_tweets, len(company_to_tweets[c][d]))
# Create index mapping for stocks alphabetically
company_to_index = {c:i for i,c in enumerate(sorted(list(company_to_tweets.keys())))}
# print dimensions
print(n_stocks)
print(n_days)
print(max_tweets)

In [ ]:
# Construct tensors
price_tensor = np.zeros((n_stocks, 1, 6, 3))
smi_tensor = np.zeros((n_stocks, 1, 6, max_tweets, 512))

In [ ]:
# SMI tensor
for company in tqdm_notebook(company_to_price_df.keys()):
    dates = sorted(list(company_to_price_df[company].date))
    lags = []
    for date in dates[:5]:
        entry = []
        row = company_to_price_df[company].loc[company_to_price_df[company]['date'] == date]
        entry.append(row['high'].values[0])
        entry.append(row['low'].values[0])
        entry.append(row['adjust_close'].values[0])
        lags.append(entry)
    for date in dates[5:6]:
        entry = []
        row = company_to_price_df[company].loc[company_to_price_df[company]['date'] == date]
        entry.append(row['high'].values[0])
        entry.append(row['low'].values[0])
        entry.append(row['adjust_close'].values[0])
        lags.append(entry)
        company_index = company_to_index[company]
        date_index = date_to_index[date]
        for i,entry in enumerate(lags):
            for j,price in enumerate(entry):
                #stocks, day, lags, hi/lo/close
                price_tensor[company_index, date_index, i, j] = price
        lags.pop(0)

In [ ]:
# SMI tensor
for company in tqdm_notebook(company_to_tweets.keys()):
    dates = sorted(list(company_to_tweets[company].keys()))
    lags = []
    for date in dates[:5]:
        n_tweets = len(company_to_tweets[company][date])
        lags.append([company_to_tweets[company][date][k]['embedding'] for k in range(n_tweets)])
    for date in dates[5:6]:
        n_tweets = len(company_to_tweets[company][date])
        lags.append([company_to_tweets[company][date][k]['embedding'] for k in range(n_tweets)])
        company_index = company_to_index[company]
        date_index = date_to_index[date]
        for i,messages in enumerate(lags):
            for j,embedding in enumerate(messages):
                #stocks, day, lags, tweet, embedding
                smi_tensor[company_index, date_index, i, j, :] = embedding[:]
        lags.pop(0)

#Testing

In [40]:
p = torch.tensor([[[1.5, 1.8, 1.2], [1.7, 1.9, 1.3]], [[0.8, 0.9, 0.6], [0.6, 0.7, 0.4]]])

In [41]:
print(p.shape)

torch.Size([2, 2, 3])


In [42]:
m = [torch.zeros(2,1,64),torch.zeros(2,1,64)]

In [43]:
print(m[0].shape)

torch.Size([2, 1, 64])


In [44]:
neighborhoods = [[0,1],[0,1,]]

In [46]:
Y = torch.tensor([[1.0],[0.0]])

In [47]:
print(Y.shape)

torch.Size([2, 1])


In [50]:
mansf = MANSF(T=2,
              num_stocks=2,
              gru_hidden_size=64,
              attn_inter_size=32,
              use_embed_size=64,
              blend_size=32,
              gat_1_inter_size=32,
              gat_2_inter_size=32,
              leakyrelu_slope=0.01,
              elu_alpha=1.0,
              U=8)

In [51]:
y = mansf(p, m, neighborhoods)

tensor([[ 0.0281, -0.0294,  0.0129, -0.0423, -0.0087, -0.0358, -0.0134,  0.0651,
         -0.0558, -0.0388,  0.0100,  0.0291,  0.0628, -0.0675,  0.0131,  0.0747,
          0.0790,  0.0087, -0.0190, -0.0334, -0.0007, -0.0370, -0.0163, -0.0152,
         -0.0352, -0.0133, -0.1151,  0.0095,  0.0727,  0.0009,  0.0278, -0.0074,
          0.0281, -0.0294,  0.0129, -0.0423, -0.0087, -0.0358, -0.0134,  0.0651,
         -0.0558, -0.0388,  0.0100,  0.0291,  0.0628, -0.0675,  0.0131,  0.0747,
          0.0790,  0.0087, -0.0190, -0.0334, -0.0007, -0.0370, -0.0163, -0.0152,
         -0.0352, -0.0133, -0.1151,  0.0095,  0.0727,  0.0009,  0.0278, -0.0074],
        [ 0.0281, -0.0294,  0.0129, -0.0423, -0.0087, -0.0358, -0.0134,  0.0651,
         -0.0558, -0.0388,  0.0100,  0.0291,  0.0628, -0.0675,  0.0131,  0.0747,
          0.0790,  0.0087, -0.0190, -0.0334, -0.0007, -0.0370, -0.0163, -0.0152,
         -0.0352, -0.0133, -0.1151,  0.0095,  0.0727,  0.0009,  0.0278, -0.0074,
          0.0207, -0.0220, 

In [52]:
print(y)  # initial predictions without training

tensor([[0.5187],
        [0.5187]], grad_fn=<SigmoidBackward>)


In [56]:
optimizer = optim.Adam(mansf.parameters())
loss_fn = nn.BCELoss()

for epoch in range(100):
    mansf.train()
    y = mansf(p, m, neighborhoods)
    loss = loss_fn(y.view(-1), Y.view(-1))
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    print(y[0].item(), y[1].item())

Streaming output truncated to the last 5000 lines.
          0.0145,  0.0380,  0.0064, -0.0634,  0.0321, -0.0538,  0.0445,  0.1073,
         -0.0465,  0.0325, -0.0034,  0.0118, -0.0424,  0.0222,  0.0570, -0.0794]],
       grad_fn=<CatBackward>)
tensor([[ 0.0561,  0.0356, -0.0375,  0.0964,  0.0467, -0.0624,  0.0635, -0.0480,
          0.0320, -0.0228, -0.0845, -0.0247, -0.0194,  0.1052, -0.0590, -0.0567,
         -0.0302,  0.1360, -0.0061, -0.0425,  0.0266, -0.0051, -0.0442,  0.0481,
         -0.0013, -0.0130,  0.0443, -0.0697, -0.1053,  0.0324, -0.0920, -0.0152,
          0.0561,  0.0356, -0.0375,  0.0964,  0.0467, -0.0624,  0.0635, -0.0480,
          0.0320, -0.0228, -0.0845, -0.0247, -0.0194,  0.1052, -0.0590, -0.0567,
         -0.0302,  0.1360, -0.0061, -0.0425,  0.0266, -0.0051, -0.0442,  0.0481,
         -0.0013, -0.0130,  0.0443, -0.0697, -0.1053,  0.0324, -0.0920, -0.0152],
        [ 0.0561,  0.0356, -0.0375,  0.0964,  0.0467, -0.0624,  0.0635, -0.0480,
          0.0320, -0.0228

In [54]:
print(y)

tensor([[0.5187],
        [0.5187]], grad_fn=<SigmoidBackward>)


In [ ]:
p_test = torch.tensor([[[1.3, 1.6, 1.0], [1.4, 1.5, 1.2]], [[0.83, 0.91, 0.67], [0.75, 0.89, 0.56]]])

In [ ]:
m_test = [torch.zeros(2,1,64),torch.zeros(2,1,64)]

In [ ]:
y_test = mansf(p_test, m_test, neighborhoods)

In [ ]:
print(y_test)

In [ ]:
p_test2 = torch.tensor([[[0.83, 0.91, 0.67], [0.75, 0.89, 0.56]], [[1.3, 1.6, 1.0], [1.4, 1.5, 1.2]]])

In [ ]:
m_test2 = [torch.zeros(2,1,64),torch.zeros(2,1,64)]

In [ ]:
y_test2 = mansf(p_test2, m_test2, neighborhoods)

In [ ]:
print(y_test2)

#Sandbox

##Price Encoding

In [ ]:
T = 5  # number of days in lookback window
batch_size = 4
gru_p_hidden_size = 64

p = torch.rand(batch_size, T, 3)  # p_i = [p_i^c, p_i^h, p_i^l], not bothering to normalize for shape tests

print('p.shape', p.shape)

h_p_0 = torch.randn(1, batch_size, gru_p_hidden_size)  # randomly initialized initial hidden state
gru_p = GRU(3, gru_p_hidden_size, batch_first=True)

h_p, h_p_n = gru_p(p, h_p_0)

print('h_p.shape', h_p.shape)
print('h_p_n.shape', h_p_n.shape)

In [ ]:
attn_p_intermediate_size = 10

attn_p = LinearAttention(gru_p_hidden_size, attn_p_intermediate_size, 1)

q = attn_p(h_p)

print('q.shape', q.shape)

##SMI Encoding

In [ ]:
K = [7, 9, 11, 13, 15]  # number of tweets for each day in lookback window
T = 5  # number or days in lookback window
batch_size = 4
gru_m_hidden_size = 64
use_embedding_size = 512

r = torch.zeros(batch_size, 0, gru_m_hidden_size)

gru_m = GRU(use_embedding_size, gru_m_hidden_size, batch_first=True)

for t in range(T):

    m = torch.rand(batch_size, K[0], use_embedding_size)

    print('m.shape', m.shape)

    h_m_0 = torch.randn(1, batch_size, gru_m_hidden_size)  # randomly initialized initial hidden state

    h_m, h_m_n = gru_m(m, h_m_0)

    print('h_m.shape', h_m.shape)
    print('h_m_n.shape', h_m_n.shape)

    attn_m_intermediate_size = 10

    attn_m = LinearAttention(gru_m_hidden_size, attn_m_intermediate_size, 1)

    r_t = attn_m(h_m)

    print('r_t.shape', r_t.shape)

    r = torch.cat((r, r_t), 1)

    print('r.shape', r.shape)

    print()

In [ ]:
gru_s_hidden_size = 64

print('r.shape', r.shape)

h_s_0 = torch.randn(1, batch_size, gru_s_hidden_size)  # randomly initialized initial hidden state
gru_s = GRU(gru_m_hidden_size, gru_s_hidden_size, batch_first=True)

h_s, h_s_n = gru_s(r, h_s_0)

print('h_s.shape', h_s_0.shape)
print('h_s_n.shape', h_s_n.shape)

In [ ]:
attn_s_intermediate_size = 10

attn_s = LinearAttention(gru_s_hidden_size, attn_s_intermediate_size, 1)

c = attn_s(h_s)

print('c.shape', c.shape)

##Blending

In [ ]:
blend_size = 9

In [ ]:
blend = Blend(q.shape[2], c.shape[2], blend_size)

In [ ]:
x = blend(q, c)

In [ ]:
print('x.shape', x.shape)

In [ ]:
num_stocks = x.shape[0]

In [ ]:
x = x.view(x.shape[0], x.shape[2])

In [ ]:
print('x.shape', x.shape)

##GAT

In [ ]:
intermediate_size = 5

In [ ]:
nhoods = [[0,1], [0,1,2], [1,2], [3]]

In [ ]:
shared_linear = SharedLinear(blend_size, intermediate_size)

In [ ]:
Wx = shared_linear(x)

In [ ]:
print('Wx.shape', Wx.shape)

##MGAT 1

In [ ]:
U = 8

In [ ]:
elu = nn.ELU()

In [ ]:
sgat = SGAT(intermediate_size, 1)

z = sgat(Wx, nhoods, 0)

z = elu(z)

for i in range(1, num_stocks):
    z_i = sgat(Wx, nhoods, i)
    z_i = elu(z_i)
    z = torch.cat((z, z_i), 0)

for u in range(1, U):
    sgat = SGAT(intermediate_size, 1)

    z_u = sgat(Wx, nhoods, 0)

    z_u = elu(z_u)

    for i in range(1, num_stocks):
        z_u_i = sgat(Wx, nhoods, i)
        z_u_i = elu(z_u_i)
        z_u = torch.cat((z_u, z_u_i), 0)
    
    z = torch.cat((z, z_u), 1)

In [ ]:
print('z.shape', z.shape)

##MGAT 2

In [ ]:
U = 8

In [ ]:
sigmoid = nn.Sigmoid()

In [ ]:
new_intermediate_size = 11

In [ ]:
shared_linear = SharedLinear(z.shape[1], new_intermediate_size)

In [ ]:
Wx = shared_linear(z)

In [ ]:
print('Wx.shape', Wx.shape)

In [ ]:
sgat = SGAT(new_intermediate_size, 1)

z = sgat(Wx, nhoods, 0)

z = sigmoid(z)

for i in range(1, num_stocks):
    z_i = sgat(Wx, nhoods, i)
    z_i = sigmoid(z_i)
    z = torch.cat((z, z_i), 0)

for u in range(1, U):
    sgat = SGAT(new_intermediate_size, 1)

    z_u = sgat(Wx, nhoods, 0)

    z_u = sigmoid(z_u)

    for i in range(1, num_stocks):
        z_u_i = sgat(Wx, nhoods, i)
        z_u_i = sigmoid(z_u_i)
        z_u = torch.cat((z_u, z_u_i), 0)
    
    z = torch.cat((z, z_u), 1)

In [ ]:
print('z.shape', z.shape)

##Final Layer

In [ ]:
linear = nn.Linear(z.shape[1], 1, bias=True)
sigmoid = nn.Sigmoid()

In [ ]:
sigmoid(linear(z))

In [ ]:
print(sigmoid(linear(z)).shape)